In [1]:
# HW 3 - NYT Articles Notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("nyt.csv")
print(df.head())
print(df.shape)

                                                text   label
0  (reuters) - carlos tevez sealed his move to ju...  sports
1  if professional pride and strong defiance can ...  sports
2  palermo, sicily — roberta vinci beat top-seede...  sports
3  spain's big two soccer teams face a pair of it...  sports
4  the argentine soccer club san lorenzo complete...  sports
(11519, 2)


Part 1: Bag Of Words (20 points):

(a) binary-valued vector

(b) frequency vector

(c) tf-idf vector

In [3]:
# Initialize Logistic Regression and split data 80-10-10

# Define X and y 
X = df["text"].tolist()
y = df["label"].tolist()

# Initialize Logistic Regression
lr = LogisticRegression()

# 90 - 10 split on entire dataset to get testing dataset (10%) and train-val dataset (90%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Split the train-val dataset (90%) into an 80 - 10 split, (80% train, 10% val)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=(10/90), random_state=42)

In [4]:
# (a)binary-valued vector
# TODO

# Initialize CountVectorizer and specify Binary = True
binary_vec = CountVectorizer(binary=True)

# Fit transform the tranining data and transform val and test data
X_train_bvec = binary_vec.fit_transform(X_train)

X_val_bvec = binary_vec.transform(X_val)
X_test_bvec = binary_vec.transform(X_test)

# Fit Linear Regression model and generate predictions based on X_val
lr.fit(X_train_bvec, y_train)
preds_bvec = lr.predict(X_val_bvec)

# Find macro f1 score (since this is binary classification)
print(f1_score(y_val, preds_bvec, average='macro'))

0.9533235897080291


In [5]:
# (b) frequency vector
# TODO

# Initialize CountVectorizer and set Binary = False for frequency/ count vectors
count_vec = CountVectorizer(binary=False)

# Fit transform the tranining data and transform val and test data
# change bvec (binary vector) to fvec (frequency vector)
X_train_fvec = count_vec.fit_transform(X_train)

X_val_fvec = count_vec.transform(X_val)
X_test_fvec = count_vec.transform(X_test)

# Fit Linear Regression model and generate predictions based on X_val
lr.fit(X_train_fvec, y_train)
preds_fvec = lr.predict(X_val_fvec)

# Find macro f1 score (since this is binary classification)
print(f1_score(y_val, preds_fvec, average='macro'))

0.9618116952759584


In [6]:
# (c) tf-idf vector
# TODO

# Initialize TfidfVectorizer, lowercase all words, and set stop words to english
tfidf_vec = TfidfVectorizer(stop_words="english", lowercase=True)

# Fit transform the tranining data and transform val and test data
# change bvec (binary vector)/ fvec (frequency vector) to tfidf
X_train_tfidf = tfidf_vec.fit_transform(X_train)

X_val_tfidf = tfidf_vec.transform(X_val)
X_test_tfidf = tfidf_vec.transform(X_test)

# Fit Linear Regression model and generate predictions based on X_val
lr.fit(X_train_tfidf, y_train)
preds_tfidf = lr.predict(X_val_tfidf)

# Find macro f1 score (since this is binary classification)
print(f1_score(y_val, preds_tfidf, average='macro'))

0.9633348981778518


Part 2: Word2Vec (20 points):

In [7]:
# TODO

# Word2Vec
from gensim.models import Word2Vec

# Tokenize sentences (lowercase and split)
def tokenize(text):
    return text.lower().split()

# Apply tokenization to X train, val and test
X_train_token = [tokenize(sentence) for sentence in X_train]
X_val_token = [tokenize(sentence) for sentence in X_val]
X_test_token = [tokenize(sentence) for sentence in X_test]

# Initialize word2vec model and set dimensions = 100
wv = Word2Vec(
    sentences=X_train_token,
    vector_size=100
)

# Iterate through every sentance in the dataset and convert into a sincle vector
# Essentially averages all word vectors into a single 100 dimension document embedding
def tokens_to_vector(token, model, size=100):
    words = [word for word in token if word in model.wv]
    if not words:
        return np.zeros(size)
    return np.mean(model.wv[words], axis=0)

# Apply token to vector onto X train, val and test
X_train_vec = np.vstack([tokens_to_vector(tokens, wv) for tokens in X_train_token])
X_val_vec   = np.vstack([tokens_to_vector(tokens, wv) for tokens in X_val_token])
X_test_vec  = np.vstack([tokens_to_vector(tokens, wv) for tokens in X_test_token])

# Train logistic regression model and calculate f1 score
lr.fit(X_train_vec, y_train)
word2vec_pred = lr.predict(X_val_vec)
print(f1_score(y_val, word2vec_pred, average='macro'))

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


0.9458014408079007


In [8]:
# GloVe

import gensim.downloader as api

# Initialize GloVe model
glove_model = api.load("glove-wiki-gigaword-100") # glove-wiki is a pretrained model, trained on wikipidia and gigword

# Convert tokens into vectors
def to_glove_vector(tokens, model, size=100):
    vectors = [model[word] for word in tokens if word in model]
    if not vectors:
        return np.zeros(size) # defualt to 0 for unknown words
    return np.mean(vectors, axis=0)
# Loops through tokenized sentences and only keeps words that exist in the GloVe model's vocab, else set as 0

# Apply conversion to X train, test, and val
X_train_vec = np.vstack([to_glove_vector(tokens, glove_model) for tokens in X_train_token])
X_val_vec   = np.vstack([to_glove_vector(tokens, glove_model) for tokens in X_val_token])
X_test_vec  = np.vstack([to_glove_vector(tokens, glove_model) for tokens in X_test_token])

#  Train logistic regression model and calculate f1 score
lr.fit(X_train_vec, y_train)
glove_pred = lr.predict(X_val_vec)
print(f1_score(y_val, glove_pred, average='macro'))

0.9416854878911732


Part 3: BERT (20 points):

In [10]:
# Perform train-test-split again but this time encode label values using label encoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])

# Define X and y 
X = df["text"].tolist()
y = df["label"].tolist()

# Initialize Logistic Regression
lr = LogisticRegression()

# 90 - 10 split on entire dataset to get testing dataset (10%) and train-val dataset (90%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Split the train-val dataset (90%) into an 80 - 10 split, (80% train, 10% val)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=(10/90), random_state=42)

In [16]:
# TODO
import torch
from transformers import AutoTokenizer
from transformers.models.bert import BertTokenizer, BertModel
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

train_data = Dataset



tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(batch):
    return tokenizer(
        batch["text"],
        trunication=True,
        padding="max_length",
        max_length=64
    )


In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

class TextData(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        tokenize = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=64,
            return_tensors="pt"
        )

        return {
            "text": text,
            "label": label,
            "input_ids": tokenize["input_ids"].squeeze(),
            "attention_mask": tokenize["attention_mask"].squeeze(),
            "labels": torch.tensor(label)
        }
    
train_data = TextData(X_train, y_train)
val_data = TextData(X_val, y_val)
test_data = TextData(X_test, y_test)


model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3
)


train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)
test_loader = DataLoader(test_data, batch_size=16)

Part 4: Summary of results / Reflection
## TODO